In [76]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [52]:
lib.utils.set_seed()
downscale = (4, 4)
test_x, test_y = lib.datasets.to_plain(*lib.datasets.get_cifar10_test(), downscale=downscale)
train_x, train_y = lib.datasets.to_plain(*lib.datasets.get_cifar10_train(), downscale=downscale)

Files already downloaded and verified
Files already downloaded and verified


In [53]:
network = lib.livenet.LiveNet.create_perceptron(train_x.shape[1], 21, 10)
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = lib.nets.criterion_n
optimizer = lib.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=100)


LiveNet


In [88]:
network.context.alpha_l1 = 0.01
trainer.step(1000)
# network.context.alpha_l1 = 0.01
# trainer.step(5000)



Iˈ0.000 3.118 = 2.880+0.237 lib/trainer.py:92
Iˈ0.670 3.114 = 2.877+0.237 lib/trainer.py:92
Iˈ1.256 3.117 = 2.874+0.243 lib/trainer.py:92
Iˈ1.828 3.115 = 2.880+0.235 lib/trainer.py:92
Iˈ2.392 3.124 = 2.883+0.240 lib/trainer.py:92
Iˈ2.957 3.121 = 2.884+0.237 lib/trainer.py:92
Iˈ3.433 killing S163->N7 at tick 9085 with k=0.001 lib/livenet.py:191
Iˈ3.434 S163 became useless at tick 9085 lib/livenet.py:61
Iˈ3.434 There is no death for S163 lib/livenet.py:44
Iˈ3.518 3.118 = 2.879+0.240 lib/trainer.py:92
Iˈ4.057 3.122 = 2.885+0.237 lib/trainer.py:92
Iˈ4.597 3.122 = 2.880+0.242 lib/trainer.py:92
Iˈ5.163 3.118 = 2.882+0.236 lib/trainer.py:92


In [89]:
pred = network(test_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.7258

In [90]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
# accum.plot()

Using matplotlib backend: TkAgg


In [91]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000.):
    print(name, f"{p.item():.1f}")

D0 -0.5
D1 -1.3
D2 1.6
D3 1.6
D4 1.6
D5 1.9
D6 2.4
D7 1.5
D8 -1.2
D9 -2.1
N0 -5.8
N3 7.0
N5 1.2
N6 2.0
N7 1.1
N9 -3.5
N10 3.8
N11 0.7
N12 -3.0
N14 -10.5
N15 -1.9
N16 1.0
N18 -2.4
N19 1.3
N0->D1 0.0
N3->D6 0.0
N5->D1 1.2
N5->D8 0.3
N5->D9 1.2
N6->D1 0.0
N7->D0 1.0
N7->D1 1.2
N7->D6 -1.1
N7->D8 0.7
N7->D9 0.5
N9->D6 0.0
N10->D7 0.0
N11->D0 1.3
N11->D8 1.9
N12->D7 0.0
N14->D7 0.0
N15->D7 0.0
N16->D5 0.0
N18->D3 0.0
N19->D5 -0.2
N19->D6 -0.2
N19->D7 0.2
N19->D9 1.0
S34->N5 0.2
S36->N5 0.4
S27->N7 -2.0
S106->N5 -0.8
S109->N5 -0.4
S115->N5 -0.5
S116->N5 -0.6
S155->N7 2.3
S156->N7 0.1
S56->N11 -0.3
S63->N11 -0.6
S16->N16 -0.0
S76->N16 0.6
S79->N16 -0.1
S83->N16 0.3
S128->N11 0.3
S139->N11 0.3
S143->N11 0.4
S148->N11 0.5
S182->N11 0.3
S123->N12 -0.1
S134->N12 0.0
S136->N16 0.1
S129->N19 0.8
S135->N19 0.9


In [19]:
network._parameters

OrderedDict([('D0',
              Parameter containing:
              tensor(-0.0146, requires_grad=True)),
             ('D1',
              Parameter containing:
              tensor(0.0146, requires_grad=True)),
             ('N4',
              Parameter containing:
              tensor(-0.0045, requires_grad=True))])

In [151]:
lib.livenet.export_onnx(network, "/home/spometun/model.onnx")

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

